In [2]:
from http.client import HTTPException
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

ModuleNotFoundError: No module named 'selenium'

In [ ]:
from time import sleep

import selenium
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup

import re

import os
import urllib
import urllib.request

import html
import sys
import random
from IPython.display import clear_output

In [ ]:
#step1: 게시물 목록 내리면서 클래스 명으로 게시물 주소 수집하기 (수집완료후 리스트 txt에 저장함)
#step2: 수집한 링크 하나씩 찾아가서 오른쪽으로 사진 넘기면서 수집하기 (gif는 수집이 현재 안됨)
#링크를 수집할 주소
URL = 'https://www.instagram.com/dlwlrma/'

CLASSNAME1 = "v1Nh3" #게시물 클래스명
CLASSNAME2 = "eLAPa" #사진 링크
CLASSNAME3 = "coreSpriteRightChevron" #오른쪽 버튼

driver = webdriver.Chrome(executable_path='./chromedriver.exe')

driver.implicitly_wait(time_to_wait=10)

In [ ]:
def check_ifClassExists(input):
    try:
        driver.find_element_by_class_name(input)
    except NoSuchElementException:
        return False
    return True

#url 언이스케이핑
def unescape(s):
    s = s.replace("&lt;", "<")
    s = s.replace("&gt;", ">")
    # this has to be last:
    s = s.replace("&amp;", "&")
    return s

In [ ]:
#인스타그램으로 이동해서 로그인하고 실행
driver.get(url=URL)
print("지정된 URL로 이동합니다...")

#스크롤 이후 기다리는 시간
SCROLL_PAUSE = 5
SCROLL_PAUSE_TWICE = 2
SCROLL_HEIGHT = 5000

#페이지 아래로 스크롤이 가능한지 확인
isBottom = False

#게시물 목록 저장하기

list_posts_links = {}

#게시글 일치 정규표현식 (게시물 목록에서 게시물로 가는 링크를 찾습니다)
p_posts = re.compile('<a href=\"\/p\/[0-9a-zA-Z\-\=\_\+]*\/\"')

#사진 일치 정규표현식 (게시물의 html에서 메인 사진의 링크를 찾습니다)
p_photos = re.compile('src="https:\/\/[a-zA-Z\-\=\+\_0-9\%\&\.\\\/\?\;]*sid\=[0-9a-zA-Z]*')



지정된 URL로 이동합니다...


In [ ]:
while (not isBottom):
    last_height = driver.execute_script("return document.body.scrollHeight")
    clear_output(wait=True)
    #게시물 링크 수집
    list_posts = driver.find_elements_by_class_name(CLASSNAME1)
    try:
        for i in range(len(list_posts)):
            
            result = p_posts.findall(list_posts[i].get_attribute('innerHTML'))
            
            result_id = result[0][12:-2] # 링크 트림 
            if (not result_id in list_posts_links):
                list_posts_links[result_id] = result

                print("링크 수집 완료: " + result_id)
    except:
        print('약간의 에러')

    #수집 후 스크롤 (다른 방법으로 스크롤할꺼면 이 부분을 주석처리)
    
    driver.execute_script("window.scrollTo(0, " + str(SCROLL_PAUSE) + ")") 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    for i in range(SCROLL_PAUSE):
        print("게시물 업데이트를 위해 기다리는 중입니다... " + str(i+1) + "초 / " + str(SCROLL_PAUSE) + "초")

    #스크롤은 셀레니움이 아니라 매크로로 진행하는게 나을듯, 자바스크립트로 내리니까 금방 차단먹음 (전 keymacro.exe로 했습니다.)

    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        sleep(SCROLL_PAUSE_TWICE)
        print("이전 높이와 지금 높이가 같습니다. 마지막 스크롤인지 체크하기 위해 대기합니다...")
        sleep(SCROLL_PAUSE_TWICE)

        # 페이지의 맨 아래 부분이면 (이전 높이와 스크롤 이후의 현재 높이가 같으면) 수집 중지
        # 게시물 수집완료 갯수가 14440개 이하면 마지막 스크롤이라 판단해도 계속 수집함 (자꾸 마지막 스크롤이라 판단하는 오류가 있어서 추가함, 필요없음)
        
        if new_height == last_height:
            if (len(list_posts_links) < 14440):
                continue
            isBottom = True
            print("마지막 스크롤입니다. 링크 수집을 종료합니다.")
    
    message = "STEP 1. 게시물 링크를 수집 중입니다. 현재 " + str(len(list_posts_links)) + "개의 게시물 링크를 수집했습니다. HEIGHT " + str(last_height) + ' / ' + str(new_height)
    print(message)
    sleep(1)
    #주소를 네이버로 바꾸면 수집을 종료함
    if driver.current_url == 'https://www.naver.com/':
        print("링크 수집 종료 오버라이드")
        break
    #주소를 다음으로 바꾸면 수집을 종료하고 링크 목록을 작성함
    if driver.current_url == 'https://www.daum.net/':
        print("현재 링크 작성 오버라이드")
        f_1 = open('insta_posts_links.txt', 'a', encoding='UTF-8')
        for i in list_posts_links:
            f_1.write(str(i)+ "\n")

In [ ]:
#수집한 링크를 파일에 저장
print("링크 수집 완료")
f_posts = open('insta_posts_links.txt', 'w', encoding='UTF-8')
print("파일 열기 완료")
for i in list_posts_links:
    f_posts.write(str(i)+ "\n")

f_posts.close()
print("파일 작성 완료")
sleep(5)
print("STEP 2. 각 게시물에 접속해서 사진을 수집합니다.")

링크 수집 완료
파일 열기 완료
파일 작성 완료
STEP 2. 각 게시물에 접속해서 사진을 수집합니다.


In [ ]:
import requests
from io import BytesIO
from PIL import Image
import time

from selenium.common.exceptions import WebDriverException
from IPython.display import clear_output

In [ ]:
#리스트의 a번째부터 b번째까지 수집하기 (계속 수집하면 아이디 차단먹기 때문에 저는 1000개하고 아이디바꾸고 1000개하고 아이디바꾸고 했습니다. (0~1000하고, 1000~2000하는식으로))
NUM_START = 0
NUM_END = 15000

#수집완료한 링크수, 에러난 링크수 초기화
linkCount = 0
errorCount = 0

driver.implicitly_wait(time_to_wait=2)

In [ ]:
#셀레니움이라는거 숨기기 위한 헤더 변경
header = {
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
"Accept": "text_html,application_xhtml+xml,application_xml;q=0.9,image_webp,**/**;q=0.8"
}

In [ ]:
#게시물 주소를 저장한 파일을 리스트로 읽기
with open("insta_posts_links.txt") as f:
    writing_links = f.readlines()
for num in range(len(writing_links)):
    writing_links[num] = writing_links[num].strip()

In [ ]:
errorCount = 0
step2Count = 1

for i in range(len(list_posts_links)):
    #위에서 지정한 번호 내의 링크만 수집하기
    if i < NUM_START:
        clear_output(wait=True)
        continue
    if i >= NUM_END:
        clear_output(wait=True)
        break

    print(str(len(writing_links)) + "개의 링크 중 " + str(i+1) + '번째 링크 수집중, ' + writing_links[i] + ", 오류 카운트: " + str(errorCount))
    urltemp = 'https://www.instagram.com/p/' + writing_links[i]

    while True:
        try:
            driver.get(urltemp)
            break
        except WebDriverException:
            sleep(5)
            continue


    print('url 이동: ' + urltemp)
    sleep(2+2*random.random())
    
    #현재 사진에서 더 넘길게 없는지 여부
    isPhotoEnd = False
    
    photoCount = 1
    tempDic = {}
    
    while (not isPhotoEnd):
        imgUrl = ""

        elem = driver.find_elements_by_class_name(CLASSNAME2)
        if (len(elem)) == 0:
            print("엘리먼트가 없습니다")

        for k in range(len(elem)):
            try:
                result2 = p_photos.findall(elem[k].get_attribute('innerHTML'))
                result2_match = result2[0][5:] # 링크 트림 
                result2_match = unescape(result2_match)
                result2_id = result2_match[-30:]
                print("현재 아이디: " + result2_id) #아이디는 각 사진의 링크인데, 이전 사진과 동일한 사진인지 여부를 확인하기 위해(마지막 사진인지 여부) 딕셔너리에 넣습니다.
                if (not result2_id in tempDic):
                    tempDic[result2_id] = result2_match
                    imgUrl = result2_match
                    #print("다릅니다")
                    break
                else:
                    #print("같습니다")
                    a=1
            except:
                print("ERROR")
                errorCount = errorCount + 1
                print("Unexpected error:", sys.exc_info()[0])
                break

            k = k + 1

        fileName = './img/' + str(writing_links[i]) + "_" + str(photoCount) + r'.jpg'
        photoCount = photoCount + 1
        print(str(photoCount-1) + "번째 이미지 다운 시도중...")
        try:
            fileName = fileName.strip()
            print("파일이름: " + fileName)
            print("url: " + imgUrl)

            res = requests.get(imgUrl, headers=header)
            img = Image.open(BytesIO(res.content))

            img.save(fileName, quality=98)

            print(str(photoCount - 1) + '번째 사진 저장 완료... FILE: ' + fileName)
        except HTTPException:
            print("다운 실패 1 : HTTP 오류")
        except:
            print("다운 실패 2, 실패 이미지 URL:")
            print(imgUrl)
            errorCount = errorCount + 1
            print("Unexpected error:", sys.exc_info()[0])
        

        if check_ifClassExists(CLASSNAME3) == False:
            isPhotoEnd = True
            print("사진의 끝입니다. 총 " + str(photoCount-1) + "개의 사진이 저장되었습니다.")
        else:
            elem = driver.find_element_by_class_name('coreSpriteRightChevron')
            elem.click()
        
        if driver.current_url == 'https://www.instagram.com/accounts/login/':
            driver.delete_all_cookies()

        sleep(1+3*random.random())
        
    clear_output(wait=True)
    step2Count = step2Count + 1